# FashionMNIST

In [2]:
import torch
from torch import nn

from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
batch_size=64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    pass

In [5]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [6]:
class NeuralNetwork(nn.Module):
	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28 * 28, 512),
			nn.ReLU(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)

	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss {loss}, {current}/{size}")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            correct += (pred.argmax(1) == y).sum().item() # changed
    
    accuracy = correct / size
    print(f"Accuracy {accuracy * 100}%")


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)

epochs = 25
for e in range(1, epochs + 1):
    print(f"Epoch {e}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
Loss 2.3020429611206055, 64/60000
Loss 2.2355735301971436, 6464/60000
Loss 2.116560697555542, 12864/60000
Loss 2.046175003051758, 19264/60000
Loss 1.807072639465332, 25664/60000
Loss 1.5927603244781494, 32064/60000
Loss 1.477236270904541, 38464/60000
Loss 1.277694582939148, 44864/60000
Loss 1.2439918518066406, 51264/60000
Loss 1.1137232780456543, 57664/60000
Accuracy 64.0%
Epoch 2
Loss 1.1698213815689087, 64/60000
Loss 1.1216623783111572, 6464/60000
Loss 0.8906809687614441, 12864/60000
Loss 1.020061731338501, 19264/60000
Loss 0.8669078350067139, 25664/60000
Loss 0.8494848608970642, 32064/60000
Loss 0.9061334133148193, 38464/60000
Loss 0.8234967589378357, 44864/60000
Loss 0.8455162048339844, 51264/60000
Loss 0.815065860748291, 57664/60000
Accuracy 70.67999999999999%
Epoch 3
Loss 0.7999089956283569, 64/60000
Loss 0.872059166431427, 6464/60000
Loss 0.6142821311950684, 12864/60000
Loss 0.829664945602417, 19264/60000
Loss 0.7079451084136963, 25664/60000
Loss 0.6779768466949463, 3206

In [8]:
torch.save(model.state_dict(), "models/model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("models/model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [9]:
classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    x = x.to(device)

    pred = model(x)
    predicted, actual = classes[pred.argmax(1)], classes[y]
    
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
